In [14]:
import pandas as pd
import numpy as np
import copy
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense,Dropout,BatchNormalization
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process.kernels import Matern
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.linear_model import LinearRegression,SGDRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from keras import layers
import matplotlib.pyplot as plt

In [21]:
data = pd.read_excel("Dataset.xlsx",header=None)
X = data.iloc[:,6:10].values
Y = data.iloc[:,4].values
Y = Y.reshape(Y.shape[0],1)

In [22]:
train_df=pd.DataFrame(X)
train_df[4]=Y
train_df

,0,1,2,3,4
0,140,60,550,10,3.1
1,140,60,550,15,3.2
2,140,60,550,20,4.1
3,140,60,650,10,3.8
4,140,60,650,15,3.6
...,...,...,...,...,...
76,160,70,650,15,3.6
77,160,70,650,20,2.9
78,160,70,750,10,3.2
79,160,70,750,15,3.9


In [23]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_train = scaler.fit_transform(train_df)

# Print out the adjustment that the scaler applied to the data
print("Note: Median values were scaled by multiplying by {:.10f} and adding {:.6f}".format(scaler.min_[4], scaler.scale_[4]))
multiplied_by = scaler.min_[4]
added = scaler.scale_[4]

scaled_train_df = pd.DataFrame(scaled_train, columns=train_df.columns.values)
scaled_train_df

Note: Median values were scaled by multiplying by -0.7419354839 and adding 0.322581


,0,1,2,3,4
0,0.0,0.0,0.0,0.0,0.258065
1,0.0,0.0,0.0,0.5,0.290323
2,0.0,0.0,0.0,1.0,0.580645
3,0.0,0.0,0.5,0.0,0.483871
4,0.0,0.0,0.5,0.5,0.419355
...,...,...,...,...,...
76,1.0,1.0,0.5,0.5,0.419355
77,1.0,1.0,0.5,1.0,0.193548
78,1.0,1.0,1.0,0.0,0.290323
79,1.0,1.0,1.0,0.5,0.516129


In [25]:
X = scaled_train_df[scaled_train_df.columns[:4]].values
Y = scaled_train_df[scaled_train_df.columns[4]].values

In [26]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.3,random_state = 0)

In [29]:
model = Sequential()
model.add(Dense(50, input_dim=4, activation='relu'))
model.add(Dense(40, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
op=Adam(learning_rate=0.001)
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [30]:
model.fit(
    X_train,
    Y_train,
    epochs=200,
    shuffle=True,
    verbose=1
)

Epoch 1/200
2/2 [==============================] - 1s 2ms/step - loss: 0.0691 - accuracy: 0.0342
Epoch 2/200
2/2 [==============================] - 0s 2ms/step - loss: 0.0714 - accuracy: 0.0446
Epoch 3/200
2/2 [==============================] - 0s 3ms/step - loss: 0.0638 - accuracy: 0.0342
Epoch 4/200
2/2 [==============================] - 0s 4ms/step - loss: 0.0605 - accuracy: 0.0342
Epoch 5/200
2/2 [==============================] - 0s 2ms/step - loss: 0.0564 - accuracy: 0.0342
Epoch 6/200
2/2 [==============================] - 0s 3ms/step - loss: 0.0553 - accuracy: 0.0342
Epoch 7/200
2/2 [==============================] - 0s 3ms/step - loss: 0.0538 - accuracy: 0.0342
Epoch 8/200
2/2 [==============================] - 0s 3ms/step - loss: 0.0593 - accuracy: 0.0446
Epoch 9/200
2/2 [==============================] - 0s 4ms/step - loss: 0.0559 - accuracy: 0.0238
Epoch 10/200
2/2 [==============================] - 0s 2ms/step - loss: 0.0518 - accuracy: 0.0461
Epoch 11/200
2/2 [===========

In [31]:
regr = make_pipeline(StandardScaler(), LinearRegression())
regr.fit(X_train,Y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])

In [32]:
regr.score(X_test,Y_test)

0.017682503322343668

In [33]:
Y_pred = regr.predict(X_test).reshape(Y_test.shape[0],1)

In [34]:
print(Y_pred)
print('\n********************************************************\n')
print(Y_test)

[[0.39299442]
 [0.48922035]
 [0.65297784]
 [0.48166778]
 [0.61058992]
 [0.42037831]
 [0.53998947]
 [0.55143958]
 [0.50598278]
 [0.490049  ]
 [0.45521365]
 [0.50208524]
 [0.36264275]
 [0.5907586 ]
 [0.37409286]
 [0.56585746]
 [0.55592326]
 [0.4672499 ]
 [0.54388701]
 [0.52405569]
 [0.51801902]
 [0.62111027]
 [0.42486198]
 [0.52191656]
 [0.49760156]]

********************************************************

[0.4516129  0.41935484 0.41935484 0.38709677 0.4516129  0.35483871
 0.61290323 0.38709677 0.4516129  0.35483871 0.19354839 0.4516129
 0.29032258 0.09677419 0.41935484 0.58064516 0.5483871  0.38709677
 0.87096774 0.48387097 0.51612903 0.93548387 0.41935484 0.32258065
 0.5483871 ]


In [36]:
np.sum(abs(Y_pred - Y_test)/Y_test)

306.76143642315606

In [10]:
import pickle
pickle.dump(regr, open("Linear_Model.sav", 'wb'))